In [21]:
import re

def ch(valid_strings, invalid_strings):
    if not valid_strings:
        return '^$'
    
    valid_symbols = None
    for s in valid_strings:
        symbols_here = set(char for char in s if not char.isalnum())
        if valid_symbols is None:
            valid_symbols = symbols_here
        else:
            valid_symbols &= symbols_here
    
    candidate_symbols = []
    if valid_symbols is not None:
        for sym in valid_symbols:
            if all(sym not in inv for inv in invalid_strings):
                candidate_symbols.append(sym)
    
    if candidate_symbols:
        symbol = candidate_symbols[0]
        return '^.+{}.+$'.format(re.escape(symbol))
    
    s0 = valid_strings[0]
    tokens = re.split(r'([^a-zA-Z0-9])', s0)
    tokens = [t for t in tokens if t != '']
    
    parts = []
    i = 0
    while i < len(tokens):
        if tokens[i].isalnum():
            parts.append(tokens[i])
            i += 1
        else:
            if i + 1 < len(tokens) and tokens[i+1].isalnum():
                parts.append(tokens[i] + tokens[i+1])
                i += 2
            else:
                parts.append(tokens[i])
                i += 1
    
    segs = []
    for part in parts:
        if part.isalpha():
            segs.append(r'\D+')
        elif part.isdigit():
            segs.append(r'\d+')
        else:
            if not part[0].isalnum():
                rest = part[1:]
                if rest == '':
                    segs.append(re.escape(part))
                else:
                    segs.append(re.escape(part[0]) + r'\w+')
            else:
                segs.append(r'.+')
    
    candidate_pattern = '^' + ''.join(segs) + '$'
    
    try:
        compiled = re.compile(candidate_pattern)
        if not any(compiled.fullmatch(inv_s) for inv_s in invalid_strings):
            return candidate_pattern
    except re.error:
        pass
    
    min_len = min(len(s) for s in valid_strings)
    common_prefix = ""
    for i in range(min_len):
        c = valid_strings[0][i]
        if all(s[i] == c for s in valid_strings):
            common_prefix += c
        else:
            break
    
    if common_prefix:
        if not any(s.startswith(common_prefix) for s in invalid_strings):
            if len(common_prefix) == 1 and all(len(s) > 1 for s in valid_strings):
                return '^[' + re.escape(common_prefix) + '].+$'
            else:
                return '^' + re.escape(common_prefix) + '.*$'
    
    common_suffix = ""
    for i in range(1, min_len + 1):
        c = valid_strings[0][-i]
        if all(s[-i] == c for s in valid_strings):
            common_suffix = c + common_suffix
        else:
            break
    
    if common_suffix:
        if not any(s.endswith(common_suffix) for s in invalid_strings):
            if len(common_suffix) == 1 and all(len(s) > 1 for s in valid_strings):
                return '^.+[' + re.escape(common_suffix) + ']$'
            else:
                return '^.*' + re.escape(common_suffix) + '$'
    
    all_chars = set(''.join(valid_strings))
    candidate_char = None
    for char in all_chars:
        if all(char in s for s in valid_strings) and all(char not in s for s in invalid_strings):
            candidate_char = char
            break
    
    if candidate_char:
        starts = all(s.startswith(candidate_char) for s in valid_strings)
        ends = all(s.endswith(candidate_char) for s in valid_strings)
        if starts:
            if all(len(s) > 1 for s in valid_strings):
                return '^[' + re.escape(candidate_char) + '].+$'
            else:
                return '^[' + re.escape(candidate_char) + ']$'
        elif ends:
            if all(len(s) > 1 for s in valid_strings):
                return '^.+[' + re.escape(candidate_char) + ']$'
            else:
                return '^[' + re.escape(candidate_char) + ']$'
        else:
            return '^.*[' + re.escape(candidate_char) + '].*$'
    
    s0 = valid_strings[0]
    n = len(s0)
    candidate_sub = None
    for length in range(n, 0, -1):
        for start in range(0, n - length + 1):
            substr = s0[start:start+length]
            if all(substr in s for s in valid_strings) and all(substr not in s for s in invalid_strings):
                candidate_sub = substr
                break
        if candidate_sub:
            break
    
    if candidate_sub:
        return '^.*' + re.escape(candidate_sub) + '.*$'
    
    return candidate_pattern

In [23]:
# Test Scroll 1
valid1 = ["abc", "def"]
invalid1 = ["123", "456"]
pattern1 = ch(valid1, invalid1)
print(f"Scroll 1: {pattern1}")

# Test Scroll 2
valid2 = ["aaa", "abb", "acc"]
invalid2 = ["bbb", "bcc", "bca"]
pattern2 = ch(valid2, invalid2)
print(f"Scroll 2: {pattern2}")

# Test Scroll 3
valid3 = ["abc1", "bbb1", "ccc1"]
invalid3 = ["abc", "bbb", "ccc"]
pattern3 = ch(valid3, invalid3)
print(f"Scroll 3: {pattern3}")

# Test Scroll 4
valid4 = ["abc-1", "bbb-1", "cde-1"]
invalid4 = ["abc1", "bbb1", "cde1"]
pattern4 = ch(valid4, invalid4)
print(f"Scroll 4: {pattern4}")

# Test Scroll 5
valid5 = ["foo@abc.com", "bar@def.net"]
invalid5 = ["baz@abc", "qux.com"]
pattern5 = ch(valid5, invalid5)
print(f"Scroll 5: {pattern5}")


valid6 = ["aaa", "bbb"]
invalid6 = ["aaaa", "bbbb"]
pattern6 = ch(valid6, invalid6)
print(f"Scroll 6: {pattern6}")


Scroll 1: ^\D+$
Scroll 2: ^[a].+$
Scroll 3: ^.+[1]$
Scroll 4: ^.+\-.+$
Scroll 5: ^\D+@\w+\.\w+$
Scroll 6: ^\D+$
